In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ==============================
# 🔹 Cargar dataset
# ==============================

# Ruta relativa desde el notebook actual
df = pd.read_csv("notebooks/data/merged_spa_suicidas.csv")



# ==============================
# 🔹 Lista de variables importantes
# ==============================
variables = [
    "anio", "upz", "sexo", "ciclo_vida", "nivel_educativo", "casos_spa",
    "SITIOHABITUALCONSUMO_VIVIENDA", "SITIOHABITUALCONSUMO_PARQUE",
    "SITIOHABITUALCONSUMO_EST_EDUCATIVO", "SITIOHABITUALCONSUMO_BARES_TABERNAS",
    "SITIOHABITUALCONSUMO_VIA_PUBLICA", "SITIOHABITUALCONSUMO_CASA_AMIGOS",
    "clasificacion", "casos_sui", "enfermedades_dolorosas", "maltrato_sexual",
    "muerte_familiar", "conflicto_pareja", "problemas_economicos", "esc_educ",
    "problemas_juridicos", "problemas_laborales", "suicidio_amigo",
    "pct_enfermedades_dolorosas", "pct_maltrato_sexual", "pct_muerte_familiar",
    "pct_conflicto_pareja", "pct_problemas_economicos", "pct_esc_educ",
    "pct_problemas_juridicos", "pct_problemas_laborales", "pct_suicidio_amigo",
    "pct_sitiohabitualconsumo_vivienda", "pct_sitiohabitualconsumo_parque",
    "pct_sitiohabitualconsumo_est_educativo", "pct_sitiohabitualconsumo_bares_tabernas",
    "pct_sitiohabitualconsumo_via_publica", "pct_sitiohabitualconsumo_casa_amigos"
]

# ==============================
# 🔹 Función para generar tabla resumen por columna
# ==============================
def generar_reporte_columna(df, col):
    serie = df[col]
    tipo = serie.dtype

    # Métricas básicas
    nulos = serie.isna().sum()
    total = len(serie)
    unicos = serie.nunique()
    duplicados = total - serie.drop_duplicates().shape[0]

    # Observaciones automáticas
    obs_tipo = "Correcto" if not pd.api.types.is_object_dtype(serie) else "Verificar formato de texto"
    obs_nulos = "No presenta datos faltantes" if nulos == 0 else f"Presenta {nulos} nulos"
    obs_unicos = "Cumple unicidad total" if unicos == total else f"{unicos} valores únicos"
    obs_dups = "Sin duplicados" if duplicados == 0 else f"{duplicados} duplicados encontrados"

    # Ejemplos de problemas (nulos o tipos extraños)
    if nulos > 0:
        ejemplo_problemas = serie[serie.isna()].head(3).tolist()
    elif serie.duplicated().any():
        ejemplo_problemas = serie[serie.duplicated()].head(3).tolist()
    else:
        ejemplo_problemas = "Ninguno"

    return {
        "Métrica": "Tipo de dato", "Valor": str(tipo), "Observaciones": obs_tipo
    }, {
        "Métrica": "Valores nulos", "Valor": nulos, "Observaciones": obs_nulos
    }, {
        "Métrica": "Valores únicos", "Valor": unicos, "Observaciones": obs_unicos
    }, {
        "Métrica": "Duplicados", "Valor": duplicados, "Observaciones": obs_dups
    }, {
        "Métrica": "Ejemplos de problemas", "Valor": str(ejemplo_problemas), "Observaciones": "Columna confiable" if ejemplo_problemas == "Ninguno" else "Revisar valores"
    }

# ==============================
# 🔹 Generar reporte para cada variable
# ==============================
reporte = {}

for col in variables:
    if col in df.columns:
        filas = generar_reporte_columna(df, col)
        reporte[col] = pd.DataFrame(filas)
    else:
        reporte[col] = pd.DataFrame({
            "Métrica": ["Error"],
            "Valor": ["No encontrada"],
            "Observaciones": ["La columna no está en el dataset"]
        })

# ==============================
# 🔹 Ejemplo: Mostrar reporte de una variable (ID o anio)
# ==============================
print("=== Reporte para la variable 'anio' ===")
print(reporte["anio"])

# ==============================
# 🔹 (Opcional) Exportar todos los reportes a Excel
# ==============================
with pd.ExcelWriter("reporte_calidad_datos.xlsx") as writer:
    for col, df_col in reporte.items():
        df_col.to_excel(writer, sheet_name=col[:31], index=False)


=== Reporte para la variable 'anio' ===
                 Métrica               Valor                 Observaciones
0           Tipo de dato               int64                      Correcto
1          Valores nulos                   0   No presenta datos faltantes
2         Valores únicos                  11             11 valores únicos
3             Duplicados               23552  23552 duplicados encontrados
4  Ejemplos de problemas  [2015, 2015, 2015]               Revisar valores


In [ ]:
# ==============================
# 🔹 Funciones auxiliares
# ==============================

def calcular_completitud(df):
    """Porcentaje de valores no nulos en todo el dataset."""
    total = df.size
    no_nulos = df.notna().sum().sum()
    return round((no_nulos / total) * 100, 2)

def calcular_exactitud(df):
    """Porcentaje de valores entre 0 y 1 en columnas tipo porcentaje."""
    pct_cols = [c for c in df.columns if "pct_" in c]
    if not pct_cols:
        return np.nan
    total = df[pct_cols].size
    validos = df[pct_cols].apply(lambda x: x.between(0, 1)).sum().sum()
    return round((validos / total) * 100, 2)

def calcular_consistencia(df):
    """Evalúa la consistencia de los datos según reglas de CASOS_SPA y CASOS_SUI."""
    
    # --------------------------
    # 1️⃣ REGLA CASOS_SPA == 0
    # --------------------------
    spa_vars = [
        "SITIOHABITUALCONSUMO_VIVIENDA",
        "SITIOHABITUALCONSUMO_PARQUE",
        "SITIOHABITUALCONSUMO_EST_EDUCATIVO",
        "SITIOHABITUALCONSUMO_BARES_TABERNAS",
        "SITIOHABITUALCONSUMO_VIA_PUBLICA",
        "SITIOHABITUALCONSUMO_CASA_AMIGOS",
    ]

    cond_spa_0 = df["casos_spa"] == 0
    if cond_spa_0.sum() > 0:
        incons_spa = df.loc[cond_spa_0, spa_vars].sum(axis=1) != 0
        consistencia_spa = 1 - (incons_spa.sum() / len(df[cond_spa_0]))
    else:
        consistencia_spa = 1.0  # Si no hay casos con 0, se asume consistente

    # --------------------------
    # 2️⃣ REGLA CASOS_SUI == 0
    # --------------------------
    sui_vars = [
        "enfermedades_dolorosas",
        "maltrato_sexual",
        "muerte_familiar",
        "conflicto_pareja",
        "problemas_economicos",
        "esc_educ",
        "problemas_juridicos",
        "problemas_laborales",
        "suicidio_amigo",
    ]

    cond_sui_0 = df["casos_sui"] == 0
    if cond_sui_0.sum() > 0:
        incons_sui = df.loc[cond_sui_0, sui_vars].sum(axis=1) != 0
        consistencia_sui = 1 - (incons_sui.sum() / len(df[cond_sui_0]))
    else:
        consistencia_sui = 1.0

    # --------------------------
    # 🔹 RESUMEN GLOBAL
    # --------------------------
    consistencia_global = (consistencia_spa + consistencia_sui) / 2

    # Se devuelve como porcentaje redondeado para mantener el mismo formato
    return round(consistencia_global * 100, 2)


def calcular_unicidad(df):
    """Porcentaje de registros únicos."""
    total = len(df)
    unicos = len(df.drop_duplicates())
    return round((unicos / total) * 100, 2)

def extraer_nivel_educativo_num(nivel):
    """Extrae el número inicial del nivel educativo (ej: '5. secundaria' → 5)."""
    try:
        return int(str(nivel).split('.')[0])
    except:
        return np.nan

def calcular_validez(df):
    """Evalúa coherencia entre ciclo de vida y nivel educativo."""
    df = df.copy()
    df["nivel_num"] = df["nivel_educativo"].apply(extraer_nivel_educativo_num)

    condiciones_validas = (
        ((df["ciclo_vida"] == "infancia") & (df["nivel_num"] <= 5)) |
        ((df["ciclo_vida"] == "adolescencia") & (df["nivel_num"] <= 7)) |
        (df["ciclo_vida"].isin(["juventud", "adultez", "vejez"]))
    )
    total = df["nivel_num"].notna().sum()
    validos = condiciones_validas.sum()
    return round((validos / total) * 100, 2)

def calcular_actualidad(df):
    """Porcentaje de registros entre 2015 y 2025."""
    if "anio" not in df.columns:
        return np.nan
    total = df["anio"].notna().sum()
    validos = df["anio"].between(2015, 2025).sum()
    return round((validos / total) * 100, 2)

# ==============================
# 🔹 Cálculo de métricas
# ==============================

reporte = pd.DataFrame([
    {
        "Dimensión": "Completitud",
        "Métrica": "% de valores no nulos",
        "Valor": calcular_completitud(df),
        "Meta": "> 95%",
        "Cumple": "✅" if calcular_completitud(df) >= 95 else "❌",
        "Observaciones": "No existen valores faltantes" if calcular_completitud(df) == 100 else "Algunas columnas con valores nulos"
    },
    {
        "Dimensión": "Exactitud",
        "Métrica": "% de columnas pct_ dentro de [0, 1]",
        "Valor": calcular_exactitud(df),
        "Meta": "> 99%",
        "Cumple": "✅" if calcular_exactitud(df) >= 99 else "❌",
        "Observaciones": "Algunos valores ligeramente fuera de rango" if calcular_exactitud(df) < 100 else "Todos los valores correctos"
    },
    {
        "Dimensión": "Consistencia",
        "Métrica": "Suma ≈ 1 en variables de sitio de consumo",
        "Valor": calcular_consistencia(df),
        "Meta": "> 95%",
        "Cumple": "✅" if calcular_consistencia(df) >= 95 else "❌",
        "Observaciones": "Leves diferencias en la suma de porcentajes" if calcular_consistencia(df) < 100 else "Perfectamente consistentes"
    },
    {
        "Dimensión": "Unicidad",
        "Métrica": "% de registros únicos",
        "Valor": calcular_unicidad(df),
        "Meta": "100%",
        "Cumple": "✅" if calcular_unicidad(df) == 100 else "❌",
        "Observaciones": "No hay duplicados" if calcular_unicidad(df) == 100 else "Existen filas duplicadas"
    },
    {
        "Dimensión": "Validez",
        "Métrica": "Edad ↔ Ciclo vida ↔ Nivel educativo coherentes",
        "Valor": calcular_validez(df),
        "Meta": "> 98%",
        "Cumple": "✅" if calcular_validez(df) >= 98 else "❌",
        "Observaciones": "Algunos infantes/adolescentes con niveles educativos altos" if calcular_validez(df) < 100 else "Datos válidos en todos los casos"
    },
    {
        "Dimensión": "Actualidad",
        "Métrica": "% de años entre 2015 y 2025",
        "Valor": calcular_actualidad(df),
        "Meta": "100%",
        "Cumple": "✅" if calcular_actualidad(df) == 100 else "❌",
        "Observaciones": "Todos los registros dentro del rango esperado" if calcular_actualidad(df) == 100 else "Algunos años fuera del rango"
    }
])

# ==============================
# 🔹 Mostrar reporte final
# ==============================
print("\n📊 REPORTE DE CALIDAD DE DATOS\n")
print(reporte.to_string(index=False))



📊 REPORTE DE CALIDAD DE DATOS

   Dimensión                                        Métrica  Valor  Meta Cumple                                              Observaciones
 Completitud                          % de valores no nulos 100.00 > 95%      ✅                               No existen valores faltantes
   Exactitud            % de columnas pct_ dentro de [0, 1] 100.00 > 99%      ✅                                Todos los valores correctos
Consistencia      Suma ≈ 1 en variables de sitio de consumo 100.00 > 95%      ✅                                 Perfectamente consistentes
    Unicidad                          % de registros únicos 100.00  100%      ✅                                          No hay duplicados
     Validez Edad ↔ Ciclo vida ↔ Nivel educativo coherentes  99.79 > 98%      ✅ Algunos infantes/adolescentes con niveles educativos altos
  Actualidad                    % de años entre 2015 y 2025 100.00  100%      ✅              Todos los registros dentro del rango espe